In [1]:
"""챗봇, 번역, 이미지 캡쳐에 사용되는 Seq2Seq  모델 구현"""
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# S 는 디코딩 입력의 시작을 나타내는 심볼
# E 는 끝을 나타냄
# P 는 현재 배치 데이터의 time step 크기보다 작은 경우 빈 시퀄스를 채우는 심볼
"""
예) 현재 배치 데이터의 최대 크기가 4인 경우
word -> ['w', 'o', 'r', 'd']
to -> ['t', 'o', 'P', 'P']
"""
char_arr = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz단어나무놀이소녀키스사랑']  # 알파벳소문자
num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)

# 영어를 한글로 번역하기 위한 학습데이터
seq_data = [['word', '단어'],['wood','나무'],
            ['game', '놀이'],['girl', '소녀'],['kiss', '키스'],['love', '사랑']]

In [3]:
def make_batch(seq_data):
    input_batch = []
    output_batch = []
    target_batch = []

    for seq in seq_data:
        # 인코더 셀의 입력값. 입력단어의 글자들을 한글자씩 떼어 배열로 만든다
        input = [num_dic[n] for n in seq[0]]
        # 디코더 셀의 입력값. 시작을 나타내는 S 심볼을 맨 앞에 붙인다
        output = [num_dic[n] for n in ('S' + seq[1])]
        target = [num_dic[n] for n in (seq[1] + 'E')]  #끝나는 지점 E

        input_batch.append(np.eye(dic_len)[input])
        output_batch.append(np.eye(dic_len)[output])
        target_batch.append(target)
    return input_batch, output_batch, target_batch

In [4]:
# *********
# 옵션 설정
# *********
learning_rate = 0.01
n_hidden = 128
total_epoch = 100

n_class = n_input = dic_len # 입력과 출력의 형태가 one-hot 인코딩으로 같으므로 크기도 동일함

In [5]:
# ********
# 신경망 모델 구성
# ********
enc_input = tf.placeholder(tf.float32, [None, None, n_input]) # [배치사이즈, 타임스텝, 인풋사이즈]
dec_input = tf.placeholder(tf.float32, [None, None, n_input])
targets = tf.placeholder(tf.int64, [None, None]) #  [배치사이즈, 타임스텝]

# 인코더 셀을 구성한다
with tf.variable_scope('encode'):
    enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)
    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input, dtype=tf.float32)

# 디코더 셀을 구성한다
with tf.variable_scope('decode'):
    dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)
    outputs, dec_states = tf.nn.dynamic_rnn(dec_cell,dec_input, 
                                            initial_state=enc_states,dtype=tf.float32)

model = tf.layers.dense(outputs, n_class, activation=None)
cost = tf.reduce_mean(
                      tf.nn.sparse_softmax_cross_entropy_with_logits(
                          logits=model, labels=targets))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.


In [6]:
# *****
# 신경망 모델 학습
# ****
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, output_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost],feed_dict={
        enc_input: input_batch,
        dec_input: output_batch,
        targets: target_batch})
    print('Epoch: ','%04d'  % (epoch + 1),
          'cost =', '{:.6f}'.format(loss))

print('최적화 완료')

Epoch:  0001 cost = 3.678409
Epoch:  0002 cost = 2.560664
Epoch:  0003 cost = 1.589798
Epoch:  0004 cost = 1.267077
Epoch:  0005 cost = 0.631159
Epoch:  0006 cost = 0.586109
Epoch:  0007 cost = 0.180354
Epoch:  0008 cost = 0.201056
Epoch:  0009 cost = 0.305857
Epoch:  0010 cost = 0.194725
Epoch:  0011 cost = 0.087245
Epoch:  0012 cost = 0.171708
Epoch:  0013 cost = 0.085120
Epoch:  0014 cost = 0.110539
Epoch:  0015 cost = 0.137306
Epoch:  0016 cost = 0.050225
Epoch:  0017 cost = 0.239832
Epoch:  0018 cost = 0.205266
Epoch:  0019 cost = 0.054115
Epoch:  0020 cost = 0.031957
Epoch:  0021 cost = 0.035090
Epoch:  0022 cost = 0.051150
Epoch:  0023 cost = 0.015367
Epoch:  0024 cost = 0.015688
Epoch:  0025 cost = 0.002119
Epoch:  0026 cost = 0.009464
Epoch:  0027 cost = 0.001919
Epoch:  0028 cost = 0.008552
Epoch:  0029 cost = 0.006438
Epoch:  0030 cost = 0.002160
Epoch:  0031 cost = 0.003053
Epoch:  0032 cost = 0.004374
Epoch:  0033 cost = 0.004726
Epoch:  0034 cost = 0.003360
Epoch:  0035 c

In [7]:
# ********
# 번역 테스트
# *******
def translate(word):
    seq_data = [word, 'P' * len(word) ]
    input_batch, output_batch, target_batch = make_batch([seq_data])
    prediction = tf.argmax(model, 2)
    result = sess.run(prediction,
                      feed_dict={enc_input: input_batch,
                                 dec_input: output_batch,
                                 targets: target_batch})
    decoded = [char_arr[i] for i in result[0]]
    end = decoded.index('E')
    translated = ' '.join(decoded[:end])
    return translated

print('=====번역 테스트========')
print('word ->', translate('word'))
print('love ->', translate('love'))
print('loev ->', translate('loev'))
print('girl ->', translate('girl'))
print('abcd ->', translate('abcd'))

=====번역 테스트========
word -> 단 어
love -> 사 랑
loev -> 사 랑
girl -> 소 녀


ValueError: 'E' is not in list